# PCA from Scratch on Auto Dataset

In [ ]:
import numpy as np
import pandas as pd

## Step 1: Load Data

In [ ]:
Auto = pd.read_csv('Auto.csv')
print("Original Data Head:")
print(Auto.head())

## Step 2: Clean Data
- Remove rows with missing horsepower ('?')
- Convert horsepower to numeric
- Drop 'name' column

In [ ]:
# Remove rows with missing horsepower
Auto = Auto[Auto['horsepower'] != '?']

# Convert horsepower to numeric
Auto['horsepower'] = pd.to_numeric(Auto['horsepower'])

# Drop 'name' column
if 'name' in Auto.columns:
    Auto = Auto.drop(columns=['name'])

print("Cleaned Data Shape:", Auto.shape)
print("Cleaned Data Head:")
print(Auto.head())

## Step 3: Standardize Data (Mean Centering and Scaling)
PCA is sensitive to the scale of features. We standardize the data to have mean 0 and standard deviation 1.

In [ ]:
X = Auto.values

# Mean centering
mean = np.mean(X, axis=0)
X_centered = X - mean

# Scaling by standard deviation
std = np.std(X_centered, axis=0)
std[std == 0] = 1 # Avoid division by zero
X_scaled = X_centered / std

print("Mean of scaled data (should be close to 0):\n", np.mean(X_scaled, axis=0))
print("Std of scaled data (should be close to 1):\n", np.std(X_scaled, axis=0))

## Step 4: Calculate Covariance Matrix
Formula: $Cov(X) = \frac{X^T X}{n-1}$

In [ ]:
n = X_scaled.shape[0]
cov_matrix = (X_scaled.T @ X_scaled) / (n - 1)

print("Covariance Matrix:")
print(cov_matrix)

## Step 5: Eigen Decomposition
Calculate eigenvalues and eigenvectors of the covariance matrix.

In [19]:
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

print("Eigenvalues (unsorted):\n", eigenvalues)
print("Eigenvectors (unsorted):\n", eigenvectors)

Eigenvalues (unsorted):
 [5.38962134 0.94607672 0.81371946 0.4873993  0.18329415 0.11461431
 0.0320513  0.05368377]
Eigenvectors (unsorted):
 [[-0.38586239  0.07663269  0.29228579  0.09998251 -0.74036644  0.38735165
   0.1151321   0.19588516]
 [ 0.4023885   0.13842878  0.07223935 -0.21603551 -0.48261485 -0.53092548
   0.41774679 -0.27878265]
 [ 0.41644435  0.12632499  0.07423622 -0.13581398 -0.30331627 -0.00699705
  -0.82916553  0.08422855]
 [ 0.40183594 -0.11148007  0.23605571 -0.11971643  0.08426839  0.6667096
   0.13477548 -0.53504996]
 [ 0.40157579  0.21102    -0.00089399 -0.32246785  0.13127292  0.23585961
   0.30991105  0.72202073]
 [-0.2647309   0.41690206 -0.63943514 -0.49280794 -0.09773197  0.20293343
  -0.03518826 -0.22891382]
 [-0.21386777  0.6904632   0.5871892  -0.10601968  0.30134385 -0.11002592
  -0.0542884  -0.12501506]
 [-0.27786815 -0.50150064  0.30732382 -0.74328281  0.04739508 -0.12086663
  -0.07951102  0.0345266 ]]


## Step 5.5: Eigen Decomposition (Pure Python Implementation)
Here we implement the **Jacobi Eigenvalue Algorithm** to find eigenvalues and eigenvectors using only Python's standard `math` library. This is to show all the logic without relying on `numpy.linalg`.

In [20]:
import math

def get_max_off_diagonal(A):
    n = len(A)
    max_val = 0.0
    p, q = 0, 1
    for i in range(n):
        for j in range(i + 1, n):
            if abs(A[i][j]) > max_val:
                max_val = abs(A[i][j])
                p, q = i, j
    return max_val, p, q

def jacobi_eigenvalue_algorithm(A, tol=1e-10, max_iterations=1000):
    # Make a copy of A to avoid modifying the original
    n = len(A)
    A_curr = [row[:] for row in A]
    
    # Initialize V as identity matrix
    V = [[1.0 if i == j else 0.0 for j in range(n)] for i in range(n)]
    
    for _ in range(max_iterations):
        max_val, p, q = get_max_off_diagonal(A_curr)
        if max_val < tol:
            break
            
        # Calculate rotation angle theta
        if A_curr[p][p] == A_curr[q][q]:
            theta = math.pi / 4
        else:
            theta = 0.5 * math.atan(2 * A_curr[p][q] / (A_curr[p][p] - A_curr[q][q]))
            
        c = math.cos(theta)
        s = math.sin(theta)
        
        # Update A_curr elements
        # We only need to update rows/cols p and q
        app = A_curr[p][p]
        aqq = A_curr[q][q]
        apq = A_curr[p][q]
        
        A_curr[p][p] = app * c**2 + aqq * s**2 + 2 * apq * c * s
        A_curr[q][q] = app * s**2 + aqq * c**2 - 2 * apq * c * s
        A_curr[p][q] = 0.0 
        A_curr[q][p] = 0.0
        
        for i in range(n):
            if i != p and i != q:
                a_ip = A_curr[i][p]
                a_iq = A_curr[i][q]
                A_curr[i][p] = a_ip * c + a_iq * s
                A_curr[p][i] = A_curr[i][p]
                A_curr[i][q] = -a_ip * s + a_iq * c
                A_curr[q][i] = A_curr[i][q]
        
        # Update eigenvectors V
        for i in range(n):
            v_ip = V[i][p]
            v_iq = V[i][q]
            V[i][p] = v_ip * c + v_iq * s
            V[i][q] = -v_ip * s + v_iq * c
            
    eigenvalues = [A_curr[i][i] for i in range(n)]
    return eigenvalues, V

# Convert numpy cov_matrix to list of lists
cov_matrix_list = cov_matrix.tolist()

py_eigenvalues, py_eigenvectors = jacobi_eigenvalue_algorithm(cov_matrix_list)

print("Pure Python Eigenvalues:")
print(py_eigenvalues)
print("\nPure Python Eigenvectors (First row):")
print(py_eigenvectors[0])

Pure Python Eigenvalues:
[0.18329415480341418, 5.389621336719716, 0.03205130329085711, 0.11461430965003448, 0.05368376661442294, 0.9460767227226865, 0.8137194618565299, 0.487399302398609]

Pure Python Eigenvectors (First row):
[0.740366438326446, -0.38586239128845196, -0.11513210447135101, 0.38735165304430463, 0.19588516078063287, 0.07663268820731316, 0.29228578622585855, -0.0999825057058032]


## Optional : Comparsion 
Here we are just reordering the resulting eigenvalues and the eigenvectors to show that step 5 and 5.5 give the same results. 

In [24]:
# Convert Jacobi results to numpy arrays
py_eigenvalues = np.array(py_eigenvalues)
py_eigenvectors = np.array(py_eigenvectors)

# Get indices 
sorted_indices = np.argsort(py_eigenvalues)[::-1]

# Reorder eigenvalues and eigenvectors
py_eigenvalues_sorted = py_eigenvalues[sorted_indices]
py_eigenvectors_sorted = py_eigenvectors[:, sorted_indices]

print("Jacobi Eigenvalues (sorted):")
print(py_eigenvalues_sorted)

print("\nJacobi Eigenvectors (sorted, first row):")
print(py_eigenvectors_sorted[0])


Jacobi Eigenvalues (sorted):
[5.38962134 0.94607672 0.81371946 0.4873993  0.18329415 0.11461431
 0.05368377 0.0320513 ]

Jacobi Eigenvectors (sorted, first row):
[-0.38586239  0.07663269  0.29228579 -0.09998251  0.74036644  0.38735165
  0.19588516 -0.1151321 ]


## Step 6: Sort Eigenvalues and Eigenvectors
Sort components by explained variance (eigenvalues) in descending order.

In [22]:
idx = np.argsort(eigenvalues)[::-1]
sorted_eigenvalues = eigenvalues[idx]
sorted_eigenvectors = eigenvectors[:, idx]

print("Sorted Eigenvalues:\n", sorted_eigenvalues)
print("Sorted Eigenvectors:\n", sorted_eigenvectors)

Sorted Eigenvalues:
 [5.38962134 0.94607672 0.81371946 0.4873993  0.18329415 0.11461431
 0.05368377 0.0320513 ]
Sorted Eigenvectors:
 [[-0.38586239  0.07663269  0.29228579  0.09998251 -0.74036644  0.38735165
   0.19588516  0.1151321 ]
 [ 0.4023885   0.13842878  0.07223935 -0.21603551 -0.48261485 -0.53092548
  -0.27878265  0.41774679]
 [ 0.41644435  0.12632499  0.07423622 -0.13581398 -0.30331627 -0.00699705
   0.08422855 -0.82916553]
 [ 0.40183594 -0.11148007  0.23605571 -0.11971643  0.08426839  0.6667096
  -0.53504996  0.13477548]
 [ 0.40157579  0.21102    -0.00089399 -0.32246785  0.13127292  0.23585961
   0.72202073  0.30991105]
 [-0.2647309   0.41690206 -0.63943514 -0.49280794 -0.09773197  0.20293343
  -0.22891382 -0.03518826]
 [-0.21386777  0.6904632   0.5871892  -0.10601968  0.30134385 -0.11002592
  -0.12501506 -0.0542884 ]
 [-0.27786815 -0.50150064  0.30732382 -0.74328281  0.04739508 -0.12086663
   0.0345266  -0.07951102]]


## Step 7: Explained Variance
Calculate the proportion of variance explained by each principal component.

In [23]:
total_variance = np.sum(sorted_eigenvalues)
explained_variance_ratio = sorted_eigenvalues / total_variance

print("Explained Variance Ratio:\n", explained_variance_ratio)
print("Cumulative Explained Variance:\n", np.cumsum(explained_variance_ratio))

Explained Variance Ratio:
 [0.67198404 0.11795791 0.10145546 0.06076949 0.02285332 0.01429024
 0.00669335 0.00399619]
Cumulative Explained Variance:
 [0.67198404 0.78994195 0.8913974  0.95216689 0.97502021 0.98931046
 0.99600381 1.        ]
